In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

In [0]:
! git clone https://github.com/GSacchetti/Segmentation-of-multispectral-and-thermal-images-with-U-NET.git

In [0]:
%cd gdrive/My Drive/U-NET

In [0]:
!ls

In [0]:
!pip install -r requirements.txt

In [0]:
!pip install xlrd==1.2

In [0]:
import xlrd

In [0]:
print(xlrd.__VERSION__)

In [0]:
%run excel_to_tif_array.py

In [0]:
%run create_train_test_data.py

In [0]:
import os
import sys
ROOT_DIR = os.path.abspath("/content/gdrive/My Drive/U-NET")
sys.path.append(ROOT_DIR)

In [0]:
%cd drive/My Drive/U-NET

In [0]:
%run main.py

In [0]:
!cat pred_unet.py

In [0]:
%run pred_unet.py